In [1]:
import pickle
import cv2
import os
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



NAMES = ["Harry","Phil","Maureen"]
numerical_labels=[]
for name in NAMES:
    try:
        index = NAMES.index(name)
        numerical_labels.append(index)
    except ValueError:
        print("Element not found")
        
print(numerical_labels)
def getFiles(NAME):
    DATABASE_PATH = "../../Labs/Datastore/Face/"+NAME+"/hasFaces"
    images=[]
    labels=[]
    MAX_IMAGES = 10
    # Loop through all files in the directory
    for filename in os.listdir(DATABASE_PATH):
        MAX_IMAGES-=1      
        if filename.endswith('.jpg'):
            # Construct the file path
            filepath = os.path.join(DATABASE_PATH, filename)
            # Read the image
            img = cv2.imread(filepath)
            ##Filtering for better recognition
                        
            images.append(img)
            labels.append(NAMES.index(NAME))
        if (MAX_IMAGES<1):
            break
    return (images,labels)


###Gather images and labels in arrays
images = []
labels = []
for name in NAMES:
    img, lab = getFiles(name)
    images+=(img)
    labels+=(lab)
    print(len(img),"Images added for",name)
print(len(images),"Images added in total ")

##hog features

def extract_hog_features(images):
    hog = cv2.HOGDescriptor()
    features = []
    for img in images:
        hog_features = hog.compute(img)
        features.append(hog_features.flatten())
    return np.array(features)

#convert face labels into numerical representations
def train_svm(features, labels):
    # Split data into training and testing subsets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Train SVM classifier
    svm_classifier = svm.SVC()
    svm_classifier.fit(X_train, y_train)

    return svm_classifier

# Step 5: Testing/Evaluation
def test_svm(svm_classifier, features, labels):
    predictions = svm_classifier.predict(features)
    accuracy = accuracy_score(labels, predictions)
    print("Accuracy:", accuracy)
    
    
# Capture new images, extract HoG features, and predict labels using the trained SVM
# Example usage
# Assuming you have face images in the 'images' list and their corresponding labels in the 'labels' list
# Convert images to grayscale if needed

print("DONE\n\nConvering images to greyscale")
gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]

# Extract HoG features
print("DONE\n\nExtracting HoG features")
hog_features = extract_hog_features(gray_images)

print("DONE\n\nTraining SVM classifier")
# Train SVM classifier
svm_classifier = train_svm(hog_features, labels)

print("DONE\n\nTesting SVM Classifier")
# Test SVM classifier
test_svm(svm_classifier, hog_features, labels)


print("\n\nSaving Classifier")
# Create the label mapping dictionary
label_mapping = {numerical_labels: label for numerical_labels, label in zip(numerical_labels, NAMES)}
# Save the SVM classifier
with open('svm_classifier.pkl', 'wb') as f:
    pickle.dump(svm_classifier, f)

# Save the label encoding mapping
with open('label_mapping.pkl', 'wb') as f:
    pickle.dump(label_mapping, f)



[0, 1, 2]
10 Images added for Harry
10 Images added for Phil
10 Images added for Maureen
30 Images added in total 
DONE
Convering images to greyscale
DONE
Extracting HoG features
DONE
Training SVM classifier
DONE
Testing SVM Classifier
